# ELT GPS Retroescavadeira
## 1. Extract
1.1. If dont have parquet file it will come from gsheet
## 2.Load
2.1. Load and convert in df pandas
## 3. Transform
3.1. Normalize de dataframe


Nestre exemplo os dados coletados em 2022 fornecidos pelo gps da retroscavadeira JCB 3CX, foram armazenados em um unico arquivo em formato de gsheet. Este estudo tem por finalidade **obter informação** utilizando tecnica da engenharia de dados

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import extrair as extrair
import fx_data as fx_data
import filtros as filtros

In [3]:
#silver
#tranformar
df = pd.read_parquet('data/bronze_jcb_relatorio_2022.parquet',engine='pyarrow')
remover_colunas = ['id','hyperlink','maps_google_url']
df1 = fx_data.df_bronze_to_silver_gps(df=df,remover_colunas=remover_colunas)

In [4]:
#salvar 
caminho = 'C:\\Users\\User\\Documents\\GitHub\\gps_maquina\\data\\' 
arquivo = 'silver_jcb_relatorio_2022.parquet'
df1.to_parquet(caminho+arquivo, index=False)

ArrowTypeError: ('Expected integer, got bool', 'Conversion failed for column motor_ligado with type object')

In [ ]:
#%%timeit
hora = 15
atividade = 'arranque_do_motor'
atividades = ['arranque_do_motor','paragem_do_motor']
consultas = [
    '(atividade == @atividade) and (hora==12)',
    '(atividade == @atividade) & (hora==12) and (nome_dia=="sunday")',
    '(atividade == @atividade) & (hora==12) or (nome_dia=="sunday") or (nome_dia=="saturday")',
    '(atividade == @atividades) & (hora==@hora)',
]

df1.query(consultas[3])

In [ ]:
lst_atividades = df1['atividade'].tolist()
def motor_ligado(atividades:list)->list:
    status =[]
    ligado=0
    for atividade in atividades:
        if atividade=='chave_ligada':
            ligado = 1
        elif atividade=='chave_desligada':
            ligado =0
        status.append(ligado)
    return status
df1['motor_ligado'] = motor_ligado(lst_atividades)

df1['lat_ant'] = df1['lat'].shift(1)

display(df1.iloc[800:810])
#len(motor_ligado(lst_atividades))
#motor_ligado(lst_atividades)

### encontrar o quando a maquina esteve em operação
1. horario
2. dia da semana

In [ ]:
def turno_atividade(atividade:str,df:pd.DataFrame)->dict:
    """
    Encontrar o horario de operação de uma atividade:   
    """
    df1 = df.query(f'atividade =="{atividade}"')
    total_linhas = df1.shape[0]
    
    turnos = {
        'magrudada':{'query':'0<=hora<=6','qtd':0,'%':0},
        'dia':{'query':'7<=hora<=17','qtd':0,'%':0},
        'noite':{'query':'18<=hora<=23','qtd':0,'%':0},
        }
    for k,v in turnos.items():
        turnos[k]['qtd'] = len(df1.query(v['query']))
        turnos[k]['%'] = round((turnos[k]['qtd']/total_linhas)*100,2)
        turnos[k]['ativo']= 1 if turnos[k]['%']>5 else 0
    
    df1['hora'].value_counts().sort_index(ascending=True).plot(kind='bar')
    
    turno_ativo = {k: v for k, v in turnos.items() if v['ativo'] > 0}
    
    return turno_ativo
turnos_arranque_motor = turno_atividade(atividade='arranque_do_motor',df=df1)
display(turnos_arranque_motor)

In [ ]:
def dia_semana_atividade(atividade:str,df:pd.DataFrame)->dict:
    df1 = df.query(f'atividade =="{atividade}"')
    total_linhas = df1.shape[0]
     
    dic = {}
    dias_semana = df1['nome_dia'].unique().tolist()
    for dia in dias_semana:
        
        dic[dia]=len(df1.query(f'nome_dia =="{dia}"'))
        dic[dia] =round(dic[dia]/total_linhas,2)
        dic[dia] = int(dic[dia]*100)
        dia_ativo = {day: value for day, value in dic.items() if value > 5}
        
        df1['nome_dia'].value_counts().sort_index(ascending=True).plot(kind='barh')
        
    return dia_ativo

dia_arranque_motor = dia_semana_atividade(atividade='arranque_do_motor',df=df1)
    
display(dia_arranque_motor)
    

### quais os locais a maquina esteve desativada

In [ ]:
df2 = df1.loc[(df1['hora'] > 20) | (df1['hora'] < 4)| (df1['atividade'] =='arranque_do_motor')]
display(df2.head(5))
unicos_lat_lon = df2['lat_lon'].unique()
display(unicos_lat_lon)
display(len(unicos_lat_lon))


### Grafico de Posicionamento x,y
Utilizando as coordenadas lat e lon e convertendo em um grafico de x,y para entender as regiões nas quais o equimamento esteve mais presente durante o periodo

In [ ]:
df1.plot.scatter(x='lon',y='lat')#, marker='o', color='blue', s=50, alpha=0.5)

# Add labels and title
#plt.xlabel('Latitude')
#plt.ylabel('Longitude')
#plt.title('Scatter Plot of Latitude vs. Longitude')

# Show the plot
#plt.show()

In [ ]:
df2.plot.scatter(x='lat', y='lon')#, marker='o', color='blue', s=50, alpha=0.5)

### calcular a distancia a partir de pontos conhecidos


In [ ]:
dic = extrair.json_to_dic('variaveis.json')
#display(dic['locais'])
df_locais = pd.DataFrame(dic['locais'])

display(df_locais)

## grafico por mes!!

In [ ]:
inicio=5
isinstance(inicio, datetime)